In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Data
# Certifique-se de que o arquivo está no mesmo diretório
df = pd.read_csv('benchmark_final_sand_2026.csv', sep=';')

# 2. Calculate Percentage Savings per Scenario
# Group by Ratio, Pod Size, and Node Size to average across executions
df_summary = df.groupby(['Ratio_R', 'TamanhoPod', 'TamanhoNode']).agg({
    'Cost_Baseline': 'mean',
    'Cost_Temporal': 'mean'
}).reset_index()

# Savings Formula
df_summary['Economy_Pct'] = ((df_summary['Cost_Baseline'] - df_summary['Cost_Temporal']) / df_summary['Cost_Baseline']) * 100

# Rename columns for better English labels in plots
df_summary = df_summary.rename(columns={'TamanhoNode': 'Node Size', 'Ratio_R': 'Ratio R'})

print("--- Savings Summary (%) ---")
print(df_summary[['Ratio R', 'TamanhoPod', 'Node Size', 'Economy_Pct']].head(15))

# 3. Visualization 1: Savings vs Ratio R
plt.figure(figsize=(10, 6))
sns.barplot(data=df_summary, x='Ratio R', y='Economy_Pct', hue='Node Size')
plt.title('Temporal Scheduler Savings vs Ratio R (Startup/Alpha)')
plt.ylabel('Average Savings (%)')
plt.xlabel('Ratio R (Startup/Alpha)')
plt.legend(title='Cluster Size (Nodes)')
plt.axhline(0, color='black', linestyle='--')
plt.savefig('savings_by_ratio_en.png')
plt.show()

# 4. Visualization 2: Temporal Behavior (Deep Dive)
# Scenario: R=100.0, 1000 Pods, 50 Nodes, Execution 1
# Renaming columns for filtering readability
df_viz = df.rename(columns={'Ratio_R': 'Ratio R', 'TamanhoPod': 'Pod Size', 'TamanhoNode': 'Node Size', 'Execucao': 'Execution'})

df_viva = df_viz[(df_viz['Ratio R'] == 100.0) & (df_viz['Pod Size'] == 1000) & (df_viz['Node Size'] == 50) & (df_viz['Execution'] == 1)]

plt.figure(figsize=(12, 6))
plt.plot(df_viva['Slot'], df_viva['Cost_Baseline'], label='Baseline (Reactive)', marker='o', color='red', linestyle='-')
plt.plot(df_viva['Slot'], df_viva['Cost_Temporal'], label='Temporal (Transition-Aware)', marker='s', color='green', linestyle='-')
plt.fill_between(df_viva['Slot'], df_viva['Cost_Temporal'], df_viva['Cost_Baseline'], color='gray', alpha=0.2, label='Cost Avoidance')

plt.title('Cost Evolution: Baseline vs Temporal (Ratio R=100)')
plt.xlabel('Time Slot')
plt.ylabel('Total Cost')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.savefig('cost_evolution_temporal_en.png')
plt.show()